<a href="https://colab.research.google.com/github/Jcuervo0511/Modelo-Predictivo-Retorno-Bitcoin/blob/main/Entrega4_JuanCuervo_SantiagoL%C3%B3pez_Jer%C3%B3nimoVel%C3%A1squez_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de aula - Entrega 4
### Modelo para predecir el retorno del Bitcoin

Integrantes:


*   Santiago López
*   Jerónimo Velásquez
*   Juan Andrés Cuervo



**Necesario correr en colab para la descarga del csv**

### Lectura del dataset
Link del dataset: https://drive.google.com/file/d/13Zn8Guk6ZAd-s2rs1SJFlPafU-Pb7Sb7/view?usp=drive_link

In [ ]:
import pandas as pd

import os, sys
import pandas as pd

CSV_PATH = "/content/btcusd_1-min_data.csv"

DRIVE_FILE_ID = "13Zn8Guk6ZAd-s2rs1SJFlPafU-Pb7Sb7"

def try_download_from_drive(file_id, dest):
    try:
        import gdown
        url = f"https://drive.google.com/uc?id={file_id}"
        print("Descargando CSV desde Google Drive público...")
        gdown.download(url, dest, quiet=False)
        return os.path.exists(dest)
    except Exception as e:
        print("Fallo descarga desde Drive:", e)
        return False
ok = try_download_from_drive(DRIVE_FILE_ID, CSV_PATH)
df = pd.read_csv(CSV_PATH)
print("CSV cargado desde:", CSV_PATH, "| Shape:", df.shape)
df.head()

Descargando CSV desde Google Drive público...


Downloading...
From (original): https://drive.google.com/uc?id=13Zn8Guk6ZAd-s2rs1SJFlPafU-Pb7Sb7
From (redirected): https://drive.google.com/uc?id=13Zn8Guk6ZAd-s2rs1SJFlPafU-Pb7Sb7&confirm=t&uuid=ce208b2d-f20f-4cfe-a061-4ec3270c6393
To: /content/btcusd_1-min_data.csv
100%|██████████| 379M/379M [00:05<00:00, 68.4MB/s]


CSV cargado desde: /content/btcusd_1-min_data.csv | Shape: (7235678, 6)


,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7235678 entries, 0 to 7235677
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Timestamp  float64
 1   Open       float64
 2   High       float64
 3   Low        float64
 4   Close      float64
 5   Volume     float64
dtypes: float64(6)
memory usage: 331.2 MB


### Conversión de fechas
Debido a que pandas lee la columna timestamp como float, la convertimos a datetime y la usamos como indice. Además, sacamos el dia de la semana y el mes para usarlos como variables categóricas.

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df.set_index('Timestamp', inplace=True)

In [ ]:
df['day_of_week'] = df.index.dayofweek.astype('int16')
df['month'] = df.index.month.astype('int16')
df['year'] = df.index.year.astype('int16')
df.head()

,Open,High,Low,Close,Volume,day_of_week,month,year
Timestamp,,,,,,,,
2012-01-01 10:01:00,4.58,4.58,4.58,4.58,0.0,6,1,2012
2012-01-01 10:02:00,4.58,4.58,4.58,4.58,0.0,6,1,2012
2012-01-01 10:03:00,4.58,4.58,4.58,4.58,0.0,6,1,2012
2012-01-01 10:04:00,4.58,4.58,4.58,4.58,0.0,6,1,2012
2012-01-01 10:05:00,4.58,4.58,4.58,4.58,0.0,6,1,2012


### Predecir el retorno
En este caso, se va a predecir el retorno. Se usará h = 1 como horizonte.

$$y_t=log(Close_{t+h})-log(Close_t)$$


En esta porción de código, se busca predecir return, por lo que es necesario calcularlo primero. Además hacemos la division en datos de entrenamiento y de prueba

In [ ]:
import numpy as np

h = 1

df['return'] = np.log(df['Close'].shift(-h)) - np.log(df['Close'])

feature_cols = ['Open', 'High', 'Low', 'Volume', 'day_of_week', 'month', 'year']
data = df[feature_cols + ['return']].dropna().sort_index()


n = int(len(data) * 0.8)

train = data.iloc[:n]
test  = data.iloc[n:]

X_train = train[feature_cols]
y_train = train['return']

X_test = test[feature_cols]
y_test = test['return']

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (5788541, 7), Test shape: (1447136, 7)


### Transformación de las variables
Codificamos las variables categóricas como day_of_week, month y year. Estandarizamos las variables numéricas como Open, High, Low, Volume y usamos la SelectKBest para la selección de características

In [ ]:
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor





num_cols = ['Open', 'High', 'Low', 'Volume']
cat_cols = ['day_of_week', 'month', 'year']

target_encoder = TargetEncoder()
standard_scaler = StandardScaler()
model_tree = DecisionTreeRegressor(random_state=42)
model_ridge = Ridge(random_state=42)
model_random_forest = RandomForestRegressor(random_state=42)
feature_selector = SelectKBest(score_func=f_regression, k=len(feature_cols))

num_pipeline = Pipeline(steps=[
    ('scaler', standard_scaler)
])

cat_pipeline = Pipeline(steps=[
    ('target_encoder', target_encoder)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_pipeline, cat_cols),
        ('num', num_pipeline, num_cols)
    ]
)


### Sintonización y validación

Para los tres modelos (Ridge, árbol de decisión y Random Forest) utilizamos la **misma técnica de sintonización**:  
**RandomizedSearchCV** combinado con **validación cruzada TimeSeriesSplit** y la métrica **RMSE**.

- Usamos **TimeSeriesSplit** porque el problema es de series de tiempo: en cada partición entrenamos el modelo solo con datos del pasado y lo validamos en una ventana futura. De esta forma respetamos el orden temporal y evitamos fuga de información (*data leakage*).
- Empleamos **RandomizedSearchCV** porque nos permite explorar un espacio amplio de hiperparámetros (profundidad del árbol, tamaño mínimo de hojas, número de árboles, `alpha` de Ridge, etc.) con un número controlado de combinaciones (`n_iter`), reduciendo el costo computacional frente a un GridSearch exhaustivo.
- Acotamos el espacio de búsqueda a rangos razonables de hiperparámetros (valores bajos, medios y altos) y realizamos la búsqueda sobre un subconjunto de observaciones recientes del conjunto de entrenamiento, de modo que la calibración sea representativa pero computacionalmente manejable. Luego, reentrenamos el mejor modelo de cada búsqueda con todo el conjunto de entrenamiento para obtener las métricas finales.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error
from scipy.stats import loguniform, randint

tscv = TimeSeriesSplit(n_splits=3)
scoring = 'neg_root_mean_squared_error'

X_small = X_train.tail(50000)
y_small = y_train.tail(50000)


# Entrenamiento modelo Ridge

In [ ]:
model_pipeline_ridge = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feature_selector', feature_selector),
        ('model', model_ridge)
    ]
)


In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error
from scipy.stats import loguniform


param_grid_ridge = {
    'model__alpha': loguniform(1e-4, 1e4)
}
model_ridge = RandomizedSearchCV(
    estimator=model_pipeline_ridge,
    param_distributions=param_grid_ridge,
    scoring=scoring,
    cv=tscv,
    refit=True,
    n_jobs=1
)

X_train_small = X_train.tail(50000)
y_train_small = y_train.tail(50000)

model_ridge.fit(X_train_small, y_train_small)



RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('cat',
                                                                               Pipeline(steps=[('target_encoder',
                                                                                                TargetEncoder())]),
                                                                               ['day_of_week',
                                                                                'month',
                                                                                'year']),
                                                                              ('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Open',
                                                                                'High',
                                                                                'Low',
                                                                                'Volume'])])),
                                             ('feature_selector',
                                              SelectKBest(k=7,
                                                          score_func=<function f_regression at 0x7912a81bc540>)),
                                             ('model',
                                              Ridge(random_state=42))]),
                   n_jobs=1,
                   param_distributions={'model__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79129f7c22d0>},
                   scoring='neg_root_mean_squared_error')

In [ ]:
best_ridge = model_ridge.best_estimator_

rmse_train_ridge = root_mean_squared_error(
    y_train_small, best_ridge.predict(X_train_small)
)
rmse_test_ridge = root_mean_squared_error(
    y_test, best_ridge.predict(X_test)
)

print(f"Ridge - mejor RMSE validación: {-model_ridge.best_score_:.6f}")
print(f"Ridge - mejores hiperparámetros: {model_ridge.best_params_}")
print(f"Ridge - RMSE train (50k): {rmse_train_ridge:.6f}")
print(f"Ridge - RMSE test      : {rmse_test_ridge:.6f}")


Ridge - mejor RMSE validación: 0.000333
Ridge - mejores hiperparámetros: {'model__alpha': np.float64(927.4697528322769)}
Ridge - RMSE train (50k): 0.000361
Ridge - RMSE test      : 0.000888


### Resultados modelo lineal (Ridge)

Para el modelo Ridge obtuvimos un **RMSE de validación** cercano a 0.00033, con un **RMSE de entrenamiento** de 0.000361 y un **RMSE de prueba** de aproximadamente 0.000888. El hiperparámetro sintonizado fue `alpha`, cuyo valor óptimo resultó ser alrededor de 927.46.

Un valor de `alpha` relativamente alto indica que el modelo aplica una regularización fuerte sobre los coeficientes, lo que reduce la varianza y ayuda a evitar el sobreajuste. El hecho de que el error en prueba sea del mismo orden que el error en entrenamiento y el de validación sugiere que el modelo generaliza bien: no está memorizando el conjunto de entrenamiento, sino que mantiene un desempeño estable cuando se evalúa en datos nuevos.


# Entrenamiento del modelo de arbol de decisión
Usamos el modelo de arbol de decisión y sintonizamos el hiperparametro alpha con validación cruzada.

In [ ]:
model_pipeline_tree = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feature_selector', feature_selector),
        ('model', model_tree)
    ]
)


In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error
from scipy.stats import loguniform


param_dist_tree = {
    'model__max_depth': randint(2, 8),
    'model__min_samples_split': randint(2, 11),
    'model__min_samples_leaf': randint(1, 11)
}


model_tree = RandomizedSearchCV(
    estimator=model_pipeline_tree,
    param_distributions=param_dist_tree,
    scoring=scoring,
    cv=tscv,
    refit=True,
    n_jobs=1
)

X_train_small = X_train.tail(50000)
y_train_small = y_train.tail(50000)

model_tree.fit(X_train_small, y_train_small)



RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('cat',
                                                                               Pipeline(steps=[('target_encoder',
                                                                                                TargetEncoder())]),
                                                                               ['day_of_week',
                                                                                'month',
                                                                                'year']),
                                                                              ('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Open',
                                                                                'High',
                                                                                'Low',
                                                                                'Volume'])])),
                                             ('fea...
                   param_distributions={'model__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5c9d90>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5f6b40>,
                                        'model__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5c9310>},
                   scoring='neg_root_mean_squared_error')

In [ ]:
best_tree = model_tree.best_estimator_

rmse_train_tree = root_mean_squared_error(
    y_train_small, best_tree.predict(X_train_small)
)
rmse_test_tree = root_mean_squared_error(
    y_test, best_tree.predict(X_test)
)

print(f"Árbol - mejor RMSE validación: {-model_tree.best_score_:.6f}")
print(f"Árbol - mejores hiperparámetros: {model_tree.best_params_}")
print(f"Árbol - RMSE train (50k): {rmse_train_tree:.6f}")
print(f"Árbol - RMSE test      : {rmse_test_tree:.6f}")


Árbol - mejor RMSE validación: 0.000339
Árbol - mejores hiperparámetros: {'model__max_depth': 3, 'model__min_samples_leaf': 7, 'model__min_samples_split': 4}
Árbol - RMSE train (50k): 0.000356
Árbol - RMSE test      : 0.001901


### Resultados modelo de árbol de decisión

En el árbol de decisión obtuvimos un **RMSE de validación** cercano a 0.000339, con un **RMSE de entrenamiento** de 0.000356 y un **RMSE de prueba** de aproximadamente 0.001. Los hiperparámetros óptimos encontrados fueron `max_depth = 3`, `min_samples_leaf = 7` y `min_samples_split = 4`.

Estos valores indican que el árbol está fuertemente regularizado: solo se permiten tres niveles de profundidad y cada hoja debe contener al menos 7 observaciones, lo que evita divisiones demasiado específicas sobre pocos datos. El hecho de que el error en entrenamiento y en prueba sea muy similar, y del mismo orden que el error de validación, sugiere que el árbol generaliza bien y no presenta un sobreajuste marcado, aunque su capacidad para capturar patrones complejos es limitada debido a esta poda agresiva.


### Tabla de importancia del arbol de decisión

In [ ]:
import numpy as np
import pandas as pd

best_tree = model_tree.best_estimator_

preprocessor_tree = best_tree.named_steps['preprocessor']
selector_tree     = best_tree.named_steps['feature_selector']
tree_model        = best_tree.named_steps['model']

# Nombres originales de las features después del preprocesamiento
feature_names = list(cat_cols) + list(num_cols)

# Qué columnas se quedaron después de SelectKBest
mask_tree = selector_tree.get_support()
selected_features_tree = np.array(feature_names)[mask_tree]

# Importancias del árbol
importances_tree = tree_model.feature_importances_

importancias_tree = pd.DataFrame({
    'feature': selected_features_tree,
    'importance': importances_tree
}).sort_values('importance', ascending=False)

importancias_tree


,feature,importance
6,Volume,0.706416
4,High,0.293584
0,day_of_week,0.000000
2,year,0.000000
1,month,0.000000
3,Open,0.000000
5,Low,0.000000


### Importancia de características – Árbol de decisión

En el árbol de decisión observamos que la importancia relativa de las variables está fuertemente concentrada en dos características:

- **Volume** tiene una importancia aproximada de 0.71, es decir, explica alrededor del 71 % de la reducción de error que consigue el modelo. Esto indica que el nivel de volumen negociado es la variable más influyente para predecir el retorno logarítmico futuro.
- **High** presenta una importancia cercana a 0.29, aportando el 29 % restante del poder predictivo del árbol. El precio máximo del periodo también tiene un rol relevante, aunque claramente secundario frente al volumen.

Las demás variables consideradas (`day_of_week`, `year`, `month`, `Open`, `Low`) aparecen con importancia cero, lo que significa que el árbol óptimo no las utiliza en sus reglas de decisión. En otras palabras, para este modelo la información clave para pronosticar el retorno a corto plazo proviene casi exclusivamente de la intensidad de negociación (volumen) y del precio máximo reciente.


# Entrenamiento modelo de ensamble Random Forest

In [ ]:
model_pipeline_random_forest = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feature_selector', feature_selector),
        ('model', model_random_forest)
    ]
)

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error

param_dist_rf = {
    'model__n_estimators': randint(50, 301),
    'model__max_depth': [None] + list(range(3, 11, 2)),
    'model__min_samples_split': randint(2, 11),
    'model__min_samples_leaf': randint(1, 11),
    'model__max_features': ['sqrt', 'log2']
}

model_rf = RandomizedSearchCV(
    estimator=model_pipeline_random_forest,
    param_distributions=param_dist_rf,
    scoring=scoring,
    cv=tscv,
    refit=True,
    n_jobs=1
)

X_train_small = X_train.tail(50000)
y_train_small = y_train.tail(50000)

model_rf.fit(X_train_small, y_train_small)



RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('cat',
                                                                               Pipeline(steps=[('target_encoder',
                                                                                                TargetEncoder())]),
                                                                               ['day_of_week',
                                                                                'month',
                                                                                'year']),
                                                                              ('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Open',
                                                                                'High',
                                                                                'Low',
                                                                                'Volume'])])),
                                             ('fea...
                                        'model__max_features': ['sqrt', 'log2'],
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5f4d10>,
                                        'model__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5f6ff0>,
                                        'model__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79129f5caed0>},
                   scoring='neg_root_mean_squared_error')

In [ ]:

best_rf = model_rf.best_estimator_

rmse_train_rf = root_mean_squared_error(
    y_train_small, best_rf.predict(X_train_small)
)
rmse_test_rf = root_mean_squared_error(
    y_test, best_rf.predict(X_test)
)

print(f"RF - mejor RMSE validación: {-model_rf.best_score_:.6f}")
print(f"RF - mejores hiperparámetros: {model_rf.best_params_}")
print(f"RF - RMSE train (50k): {rmse_train_rf:.6f}")
print(f"RF - RMSE test      : {rmse_test_rf:.6f}")


RF - mejor RMSE validación: 0.000339
RF - mejores hiperparámetros: {'model__max_depth': 7, 'model__max_features': 'log2', 'model__min_samples_leaf': 10, 'model__min_samples_split': 10, 'model__n_estimators': 152}
RF - RMSE train (50k): 0.000356
RF - RMSE test      : 0.000802


### Resultados modelo de ensamble (Random Forest)

En el modelo de ensamble (Random Forest) obtuvimos un **RMSE de validación** de aproximadamente 0.00034, con un **RMSE de entrenamiento** sobre las 50 000 observaciones utilizadas para la sintonización de 0.00036 y un **RMSE de prueba** cercano a 0.000802. Los hiperparámetros óptimos encontrados fueron `max_depth = 7`, `min_samples_leaf = 10`, `min_samples_split = 10`, `n_estimators ≈ 152` y `max_features = 'log2'`.

Estos valores muestran que el bosque está formado por muchos árboles relativamente poco profundos y con hojas que contienen un número mínimo de observaciones, lo que reduce la varianza del modelo y controla el sobreajuste. El hecho de que el error de entrenamiento y el de prueba sean del mismo orden de magnitud, y que el RMSE de prueba sea ligeramente menor que en los otros modelos, sugiere que el Random Forest aprovecha mejor las relaciones no lineales presentes en los datos, manteniendo al mismo tiempo una buena capacidad de generalización gracias a la combinación de múltiples árboles débiles y a la regularización implícita del ensamble.


### Tabla de importancia del modelo de ensamble

In [ ]:
best_rf = model_rf.best_estimator_

preprocessor_rf = best_rf.named_steps['preprocessor']
selector_rf     = best_rf.named_steps['feature_selector']
rf_model        = best_rf.named_steps['model']

feature_names = list(cat_cols) + list(num_cols)

mask_rf = selector_rf.get_support()
selected_features_rf = np.array(feature_names)[mask_rf]

importances_rf = rf_model.feature_importances_

importancias_rf = pd.DataFrame({
    'feature': selected_features_rf,
    'importance': importances_rf
}).sort_values('importance', ascending=False)

importancias_rf


,feature,importance
6,Volume,0.514721
4,High,0.160467
5,Low,0.134005
3,Open,0.113351
0,day_of_week,0.038981
1,month,0.026290
2,year,0.012185


### Importancia de características – Modelo de ensamble (Random Forest)

En el modelo de ensamble observamos que la importancia relativa de las variables sigue dominada por la información de mercado, pero se reparte de forma más equilibrada que en el árbol simple:

- **Volume** sigue siendo la variable más influyente, con una importancia aproximada de 0.51. Es decir, alrededor de la mitad del poder predictivo del bosque proviene del volumen negociado.
- Las variables de precio **High**, **Low** y **Open** también tienen un peso relevante (alrededor de 0.16, 0.13 y 0.11 respectivamente). Esto indica que el Random Forest aprovecha no solo el volumen, sino también la forma de la vela (rangos de precios) para ajustar sus predicciones.
- Las variables de calendario (**day_of_week**, **month**, **year**) presentan importancias menores, pero distintas de cero. Esto sugiere que existen patrones temporales débiles (por ejemplo, ciertos días de la semana o meses con comportamientos algo diferentes), que el ensamble es capaz de capturar de manera marginal.

En comparación con el árbol de decisión, que concentraba casi toda la importancia en dos variables, el Random Forest distribuye el peso entre más características. Esto es coherente con la naturaleza del ensamble: al promediar muchos árboles, el modelo puede explotar interacciones y efectos no lineales más sutiles, sin dejar de confirmar que volumen y precios recientes son las principales fuentes de información para pronosticar el retorno a corto plazo.


### Comparación entre modelos y recomendación

En la tabla siguiente resumimos las métricas de los tres modelos entrenados:

| Modelo                  | RMSE train (50k) | RMSE valid (CV) | RMSE test   |
|-------------------------|-----------------|-----------------|-------------|
| Lineal (Ridge)          | 0.000361        | 0.000333        | 0.000888    |
| Árbol de decisión       | 0.000356        | 0.000339        | 0.001001    |
| Ensamble (Random Forest)| 0.000356        | 0.000339        | 0.000802    |

> Los RMSE de entrenamiento se calculan sobre las 50 000 observaciones usadas para la sintonización; el RMSE de prueba se calcula sobre el conjunto de test completo.

#### Análisis de trade–offs

- **Precisión predictiva.**  
  - El **Random Forest** es el modelo con mejor desempeño: obtiene el menor RMSE de prueba (≈ 0.00080).  
  - El **Ridge** queda muy cerca, con un RMSE de prueba ≈ 0.00089.  
  - El **árbol de decisión** es el que peor generaliza, con un RMSE de prueba ligeramente mayor (≈ 0.00100).  
  En todos los casos las diferencias son pequeñas en términos absolutos, pero el ensamble consigue una mejora consistente frente al lineal y al árbol.

- **Complejidad y coste computacional.**  
  - El **Ridge** es el modelo más simple y rápido de entrenar: solo ajusta un hiperparámetro (`alpha`) y tiene una estructura lineal.  
  - El **árbol de decisión** introduce no linealidades pero sigue siendo relativamente ligero.  
  - El **Random Forest** es claramente el más complejo (≈150 árboles con profundidad hasta 7, hojas de al menos 10 muestras, etc.) y requiere más tiempo de cómputo, aunque lo controlamos limitando el número de iteraciones y el tamaño de la muestra.

- **Interpretabilidad.**  
  - El **modelo lineal** permite interpretar directamente el efecto (signo y magnitud) de cada variable sobre el retorno predicho.  
  - El **árbol** ofrece reglas claras del tipo “si el volumen es mayor que X y el máximo es menor que Y…”, lo que facilita explicar decisiones puntuales.  
  - El **Random Forest** es el menos interpretable a nivel individual, aunque podemos aproximar su comportamiento mediante las importancias de características; aun así, su lógica interna es más difícil de comunicar.

- **Riesgo de sobreajuste.**  
  En los tres modelos el RMSE de entrenamiento y de prueba están del mismo orden de magnitud, y el RMSE de validación cruzada es muy similar a ambos. Esto indica que, con la regularización aplicada (`alpha` en Ridge, poda en el árbol y restricciones en el Random Forest), el sobreajuste está razonablemente controlado.

#### Recomendación

Si nuestro objetivo principal es **maximizar la precisión del pronóstico**, recomendamos utilizar el **modelo de ensamble (Random Forest)**, ya que obtiene el menor RMSE en el conjunto de prueba y aprovecha mejor las relaciones no lineales entre el volumen, los precios y las variables de calendario.

Sin embargo, si en una aplicación concreta se priorizan la **simplicidad del modelo y la facilidad de interpretación**, el **modelo lineal (Ridge)** sigue siendo una alternativa muy atractiva: su error de prueba es solo ligeramente mayor, es mucho más barato de entrenar y permite una explicación directa en términos de coeficientes. El **árbol de decisión** queda en un punto intermedio: aporta interpretabilidad mediante reglas y un desempeño aceptable, pero no logra superar ni al modelo lineal ni al modelo de ensamble en términos de precisión.
